# Product Demand Forecasting Model

**Objective:** To forecast the next 30 days of sales for the 'Pet Food' category using historical data from Snowflake.

In [ ]:
# PROD RELEASE (2-4-22)
import pandas as pd
import snowflake.connector
import os
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

print("Libraries imported.")

In [ ]:
# [NOTEBOOK_AGENT]: SQL in this cell was automatically translated.
# Fetch historical daily sales data for the 'Pet Food' category
conn = snowflake.connector.connect(
    user=os.getenv('SNOWFLAKE_USER'),
    password=os.getenv('SNOWFLAKE_PASSWORD'),
    account=os.getenv('SNOWFLAKE_ACCOUNT'),
    database='RETAIL_PROD',
    schema='SALES'
)

query = """```sql
SELECT 
    SALE_DATE, 
    SUM(SALE_AMOUNT) as DAILY_SALES
FROM 
    `HISTORICAL_SALES`
WHERE 
    PRODUCT_CATEGORY = 'Pet Food'
GROUP BY SALE_DATE
ORDER BY SALE_DATE
```"""

daily_sales_df = pd.read_sql(query, conn)
conn.close()

print(f"Loaded {len(daily_sales_df)} days of sales data.")

In [ ]:
# Data Preparation: Create a proper time-series index
daily_sales_df['SALE_DATE'] = pd.to_datetime(daily_sales_df['SALE_DATE'])
daily_sales_df.set_index('SALE_DATE', inplace=True)

# Resample to ensure we have a value for every day, filling missing days with 0
ts_data = daily_sales_df['DAILY_SALES'].resample('D').fillna(0)

print("Time-series data prepared for modeling.")
ts_data.plot(title='Daily Pet Food Sales', figsize=(15, 6))

In [ ]:
# Model Training: Fit a simple SARIMA model
# (Note: p,d,q parameters are chosen for simplicity in this demo)
print("Training SARIMA model...")
model = SARIMAX(ts_data, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
model_fit = model.fit(disp=False)

print("Model training complete.")

In [ ]:
# Forecasting: Predict the next 30 days
print("Generating forecast for the next 30 days...")
forecast = model_fit.forecast(steps=30)

# Plot the forecast against historical data
plt.figure(figsize=(15, 7))
plt.plot(ts_data.tail(90), label='Historical Sales')
plt.plot(forecast, label='Forecasted Sales', color='red')
plt.title('Sales Forecast vs. Historical Data')
plt.legend()
plt.show()